In [2]:
import os
import json
import pandas as pd
import traceback


In [6]:
from langchain.chat_models import ChatOpenAI # langchain_community needed to be installed, manual installed it here


In [10]:
from dotenv import load_dotenv
load_dotenv()  # take environment varibles from .env


True

In [9]:
Key=os.getenv("OPENAI_API_KEY")
